# Bibtex to DF and tidy

**One off code for data rationalization and normalization.**

* df
* author list
* publisher list
* journal list
* files
* normalization

item 241 has weird annot


## Provenance
* Created new 2025-05-16 at Bignor!

In [1]:
from pathlib import Path 
import pandas as pd
import re
from functools import partial
import pprint as pp

import numpy as np
import Levenshtein
import sys
sys.path.append('..')
import archivum.mendeley_port as sc
from archivum.utilities import fGT

import great2_bibtex as scbt

%load_ext autotime

C:\Users\steve\AppData\Local\archivum


time: 94 ms (started: 2025-05-24 12:15:09 +01:00)


In [2]:
p = Path('books.bib')
p = Path('library.bib')
p.exists()

True

time: 15 ms (started: 2025-05-24 12:15:09 +01:00)


# Uber import 

In [5]:
b = sc.Bib2df(p, True)

uber regex sub found n = 517 replacements
time: 578 ms (started: 2025-05-24 12:16:52 +01:00)


In [7]:
b.port_mendeley_file()

Audit DataFrame len(df) = 8160 saved to C:\Users\steve\AppData\Local\archivum\imports\library.bib.author-mapping.utf-8-sig.csv
	Field: title	len(self.last_decode) = 37
Fixed title
	Field: journal	len(self.last_decode) = 4
Fixed journal
Fixed publisher
Fixed institution
Fixed booktitle
Fixed address
Fixed editor
Fixed mendeley-tags
Audit DataFrame len(df) = 41 saved to C:\Users\steve\AppData\Local\archivum\imports\library.bib.tex-unicode-errors.utf-8-sig.csv
Audit DataFrame len(df) = 6118 saved to C:\Users\steve\AppData\Local\archivum\imports\library.bib.tag-mapping.utf-8-sig.csv
Audit DataFrame len(df) = 6118 saved to C:\Users\steve\AppData\Local\archivum\imports\library.bib.raw-df.utf-8-sig.csv
Audit DataFrame len(df) = 6118 saved to C:\Users\steve\AppData\Local\archivum\imports\library.bib.ported-df.utf-8-sig.csv
Audit DataFrame len(df) = 4 saved to C:\Users\steve\AppData\Local\archivum\imports\library.bib.audit-info.utf-8-sig.csv
time: 4.95 s (started: 2025-05-24 12:22:06 +01:00)


In [10]:
b.ported_df.loc[b.ported_df.tag.duplicated(keep=False)]

,type,tag,author,doi,file,journal,pages,title,volume,year,...,url,institution,number,mendeley-tags,booktitle,edition,month,address,editor,arc-citations


time: 16 ms (started: 2025-05-24 12:22:59 +01:00)


In [12]:
pout = Path(r'C:\Users\steve\AppData\Local\archivum\uber-library.archivum-feather')
pout.parent.exists()

True

time: 0 ns (started: 2025-05-24 12:52:11 +01:00)


In [14]:
b.ported_df.to_feather(pout)

time: 78 ms (started: 2025-05-24 12:53:42 +01:00)


## Manual read and manual inspection

In [8]:
b = sc.Bib2df(p, False)
len(b.df)

uber regex sub found n = 517 replacements


6118

time: 563 ms (started: 2025-05-23 21:05:56 +01:00)


In [9]:
b.df.head(2).T

,1,2
type,article,book
tag,Schmidt2003,Bahnemann2015
abstract,It is shown that the arbitrage free portfolio paradigm being applied to a portfolio with an arbi...,NaN
author,"Schmidt, Anatoly B.","Bahnemann, David"
doi,10.1016/S0378-4371(02)01551-0,NaN
file,:C\:/S/Library/Schmidt/2003_The true invariant of an arbitrage free portfolio.pdf:pdf,:C\:/S/Library/Bahnemann/2015_Distributions for Actuaries.pdf:pdf;:C\:/S/Library/Bahnemann/2015_...
issn,03784371,NaN
journal,Physica A: Statistical Mechanics and its Applications,NaN
keywords,"Arbitrage free portfolio,Option pricing",NaN
pages,535--538,NaN


time: 0 ns (started: 2025-05-23 21:05:57 +01:00)


In [10]:
# filled values for each column  
b.contents()

,column,nonna,distinct
0,type,6118,10
1,tag,6118,6076
2,abstract,2518,2375
3,author,6113,4856
4,doi,2687,2538
5,file,5951,5917
6,issn,2425,826
7,journal,3475,1224
8,keywords,2999,2052
9,pages,3885,2748


time: 62 ms (started: 2025-05-23 21:06:05 +01:00)


In [ ]:
# value counts for each field to look at the data
for f in b.contents().column: 
    display(fGT(b.df[f].value_counts().sort_values()[::-1].head(30), caption=f))
    print()

In [ ]:
# distinct col c
j = b.distinct('journal')
j[:10]

# Author Names

1. remove inital punctuation
1. de-tex (unicode)
1. find longest unique extension
1. add back accents (slight hack)
1. --> proposed


In [53]:
b._author_map_df = None 
adf = b.author_map_df

Field: authors
Decode errors: len(self.last_decode) = 0
time: 1.17 s (started: 2025-05-23 21:41:00 +01:00)


In [55]:
adf.iloc[20:25]

,original,unicoded,longest,accents,proposed
20,"Abbott, B P","Abbott, B P","Abbott, B P","Abbott, B P","Abbott, B. P."
21,"Abdel-Hameed, Mohamed","Abdel-Hameed, Mohamed","Abdel-Hameed, Mohamed","Abdel-Hameed, Mohamed","Abdel-Hameed, Mohamed"
22,"Abdikerimova, Samal","Abdikerimova, Samal","Abdikerimova, Samal","Abdikerimova, Samal","Abdikerimova, Samal"
23,"Abowd, John M.","Abowd, John M","Abowd, John M","Abowd, John M","Abowd, John M."
24,"Abraham, Andreas","Abraham, Andreas","Abraham, Andreas","Abraham, Andreas","Abraham, Andreas"


time: 15 ms (started: 2025-05-23 21:41:05 +01:00)


In [57]:
len(adf.query('longest != unicoded'))

409

time: 0 ns (started: 2025-05-23 21:41:34 +01:00)


In [58]:
adf.query('longest != unicoded').sample(20)

,original,unicoded,longest,accents,proposed
2383,"Ghossoub, M","Ghossoub, M","Ghossoub, Mario","Ghossoub, Mario","Ghossoub, Mario"
3828,"Lai, Gene","Lai, Gene","Lai, Gene C","Lai, Gene C","Lai, Gene C."
1518,"Dan{\'{i}}elsson, J","Daníelsson, J","Daníelsson, Jón","Daníelsson, Jón","Daníelsson, Jón"
1123,"Chen, John","Chen, John","Chen, John Donghui","Chen, John Donghui","Chen, John Donghui"
1656,"Diamond, Douglas W.","Diamond, Douglas W","Diamond, Douglas W","Diamond, Douglas W","Diamond, Douglas W."
13,"Aas, K","Aas, K","Aas, Kjersti","Aas, Kjersti","Aas, Kjersti"
1112,"Chatterjee, S.","Chatterjee, S","Chatterjee, Sourav","Chatterjee, Sourav","Chatterjee, Sourav"
8132,"{Van Loan}, Charles","{Van Loan}, Charles","{Van Loan}, Charles F","{Van Loan}, Charles F","{Van Loan}, Charles F."
3239,"Jiao, Z","Jiao, Z","Jiao, Zhanyi","Jiao, Zhanyi","Jiao, Zhanyi"
6550,"Spielman, Daniel","Spielman, Daniel","Spielman, Daniel A","Spielman, Daniel A","Spielman, Daniel A."


time: 15 ms (started: 2025-05-23 21:42:10 +01:00)


In [60]:
adf.query('longest != accents').sample(20)

,original,unicoded,longest,accents,proposed
103,"Albrecher, Hansjorg","Albrecher, Hansjorg","Albrecher, Hansjorg","Albrecher, Hansjörg","Albrecher, Hansjörg"
977,"Candes, Emmanuel J.","Candes, Emmanuel J","Candes, Emmanuel J","Candès, Emmanuel J","Candès, Emmanuel J."
2634,"Guillen, Montserrat","Guillen, Montserrat","Guillen, Montserrat","Guillén, Montserrat","Guillén, Montserrat"
1513,"Danielsson, Jon","Danielsson, Jon","Danielsson, Jon","Daníelsson, Jón","Daníelsson, Jón"
3986,"Letac, Gerard","Letac, Gerard","Letac, Gerard","Letac, Gérard","Letac, Gérard"
86,"Ait-Sahalia, Yacine","Ait-Sahalia, Yacine","Ait-Sahalia, Yacine","Aït-Sahalia, Yacine","Aït-Sahalia, Yacine"
3692,"Kratschmer, Volker","Kratschmer, Volker","Kratschmer, Volker","Krätschmer, Volker","Krätschmer, Volker"
5959,"Ruszczynski, Andrzej","Ruszczynski, Andrzej","Ruszczynski, Andrzej","Ruszczyński, Andrzej","Ruszczyński, Andrzej"
1116,"Chavez-Demoulin, Valerie","Chavez-Demoulin, Valerie","Chavez-Demoulin, Valerie","Chavez-Demoulin, Valérie","Chavez-Demoulin, Valérie"
2005,"Farach-Colton, Martin","Farach-Colton, Martin","Farach-Colton, Martin","Farach-Colton, Martín","Farach-Colton, Martín"


time: 16 ms (started: 2025-05-23 21:42:44 +01:00)


# edition

In [76]:
for v in sorted(b.distinct('edition')):
    print(f'"{v}": "{v.title()}",')

"10": "10",
"2": "2",
"2nd": "2Nd",
"2nd Editio": "2Nd Editio",
"3": "3",
"3rd": "3Rd",
"5": "5",
"Annals of": "Annals Of",
"Enlarged": "Enlarged",
"Fifth": "Fifth",
"First": "First",
"Fourth": "Fourth",
"Lecture No": "Lecture No",
"Ninth": "Ninth",
"Second": "Second",
"Second Edi": "Second Edi",
"Seventh": "Seventh",
"Sixth": "Sixth",
"Third": "Third",
"fourth": "Fourth",
time: 15 ms (started: 2025-05-23 22:04:39 +01:00)


# mendeley-tags
Just carry over

# Porting Menedeley

In [11]:
b = sc.Bib2df(p, True)

uber regex sub found n = 517 replacements
time: 578 ms (started: 2025-05-23 21:06:09 +01:00)


In [45]:
b.port_mendeley_file()

Field: authors
Decode errors: len(self.last_decode) = 0
fixed authors
Field: title
len(self.last_decode) = 37
Fixed title
Field: journal
len(self.last_decode) = 4
Fixed journal
Fixed publisher
Fixed institution
Fixed booktitle
Fixed address
Fixed editor
time: 14.9 s (started: 2025-05-21 11:41:38 +01:00)


In [ ]:
b.show_unicode_errors()

In [ ]:
b.contents(ported=True)

In [ ]:
# run fillna = False
df = b.contents(True)

In [56]:
# check some mappings
fGT(b.ported_df.loc[b.ported_df.author.str.contains('Can'), ['author', 'title']])

index,author,title
181,"Marucho, M. and Canal, C. A.Garcia and Fanchiotti, Huner",{The Landau distribution for charged particles traversing thin films}
599,"Canis, Bill",{Motor Vehicle Supply Chain: Effects of the Japanese Earthquake and Tsunami (Google eBook)}
937,"Bellare, M. and Canetti, R. and Krawczyk, H.",{Keying hash functions for message authentication}
944,"Candès, Emmanuel J.",{Mathematics of sparsity (and a few other things)}
1159,"Candès, Emmanuel J.",{Compressive sampling}
1675,"Gangbo, Wilfrid and McCann, Robert J.",{The geometry of optimal transportation}
1727,"Cannas, Giuseppina",{A stochastic model for the sustainable investment policy in a defined benefit pension fund}
1783,Canon,{PowerShot G15 Manual}
1848,"Candès, Emmanuel J. and Wakin, M. B.",{An Introduction To Compressive Sampling}
2241,Canon,{Canon 5DSR Instruction Manual}


time: 110 ms (started: 2025-05-21 11:56:27 +01:00)


In [54]:
b.no_file()

,tag,type,author,title,year,journal,file
38,keynesgeneral,book,"Keynes, John Maynard","{General theory of employment, interest and mo...",1936,,
53,Mildenhall2018c,article,"Mildenhall, Stephen J",{Every law invariant coherent risk measure has...,2018,,
75,Eling2017,article,"Eling, Martin and Jia, Ruo",{Recent Research Developments Affecting Nonlif...,2017,Risk Management and Insurance Review,
114,daykin94,book,"Daykin, C. D. and Pentikainen, T. and Pesonen, M.",{Practical Risk Theory for Actuaries},1994,,
133,Crouhy2010,article,"Crouhy, Michel and Galai, Dan and Mark, Robert",{Risk Exposures},2010,Encyclopedia of Quantitative Finance,
...,...,...,...,...,...,...,...
5865,hull01,book,"Hiriart-Urruty, Jean-Baptiste and Lemar{\'{e}}...",{Fundamentals of Convex Analysis},2001,,
5931,eeckhoudtgs04,book,"Eeckhoudt, Louis and Gollier, Christian and Sc...",{Economic and Financial Decisions under Risk},2005,,
6006,Strain1997,book,"Strain, Robert W.",{Reinsurance},1997,,
6039,jacob02,book,"Jacob, Niels",{Pseduo Differential Operators & Markov Proces...,2002,,


time: 32 ms (started: 2025-05-21 11:50:59 +01:00)


# Duplicated tags

In [299]:
b.port_mendeley_file()

Audit DataFrame len(df) = 8160 saved to C:\Users\steve\AppData\Local\archivum\library.bib.utf-8-sig.author-mapping.csv
	Field: title	len(self.last_decode) = 37
Fixed title
	Field: journal	len(self.last_decode) = 4
Fixed journal
Fixed publisher
Fixed institution
Fixed booktitle
Fixed address
Fixed editor
Fixed mendeley-tags
Audit DataFrame len(df) = 41 saved to C:\Users\steve\AppData\Local\archivum\library.bib.utf-8-sig.tex-unicode-errors.csv
Audit DataFrame len(df) = 6118 saved to C:\Users\steve\AppData\Local\archivum\library.bib.utf-8-sig.tag-mapping.csv
Audit DataFrame len(df) = 0 saved to C:\Users\steve\AppData\Local\archivum\library.bib.utf-8-sig.audit-info.csv
time: 4.44 s (started: 2025-05-24 11:59:00 +01:00)


In [292]:
ans = []
for k, v in b.all_unicode_errors.items():
    for mc in v:
        ans.append([k, mc])

time: 0 ns (started: 2025-05-24 11:53:35 +01:00)


'2025-05-24 12:08:59.157313'

time: 0 ns (started: 2025-05-24 12:08:59 +01:00)


In [295]:
pd.DataFrame(ans, columns=['field', 'miscode'])

,field,miscode
0,title,{Polar factorization and monotone rearrangement of vector‐valued functions}
1,title,{Buffered probability of exceedance: Mathematical properties and optimization∗}
2,title,"{R(5,5)≤48}"
3,title,{The Peter J . Tobin College of Business Professional ­ Accreditation}
4,title,markets with adverse selection ∗
5,title,{ENTROPIES AND CROSS-ENTROPIES OF EXPONENTIAL FAMILIES Ecole Polytechnique Sony Computer Science...
6,title,{Portfolio diversification and value at risk under thick-tailedness†}
7,title,{Jumps and Betas : A New Framework for Disentangling and Estimating Systematic Risks ∗}
8,title,"{Managerial Ability, Compensation, and the Closed‐End Fund Discount}"
9,title,{Near‐miss telematics in motor insurance}


time: 15 ms (started: 2025-05-24 11:53:59 +01:00)


In [284]:
b.bibtex_file_path = p
df = b.map_tags('ported_df')

time: 140 ms (started: 2025-05-24 11:38:13 +01:00)


In [286]:
fGT(df.head(100))

index,author,editor,year,tag,title,standard_tag,proposed_tag
3710,{451 Research},,2014,451Research2014,{Data Platforms Map},451Research2014,451Research2014
1399,{451 Research},,2015,451Research2015,{Data Platform Map},451Research2015,451Research2015
1946,"ACIA, Dublin",,2014,ACIA2014,{Python Notes / Cheat Sheet},ACIA2014,ACIA2014
5712,"ACIA, Dublin",,2014,ACIA2014a,{Introductory Notes : Matplotlib},ACIA2014,ACIA2014a
1795,"ACIA, Dublin",,2015,ACIA2015,{Cheat Sheet : The pandas DataFrame Object},ACIA2015,ACIA2015
3745,AIPSO,,2015,AIP2015,{AIPSO Facts 2014/15},AIPSO2015,AIPSO2015
4020,AIPSO,,2016,Manuals,{AIPSO Facts 2015/16},AIPSO2016,AIPSO2016
758,{AIR Worldwide},,2009,AIRWorldwide,{AIR Earthquake Model for Chile},AIRWorldwide2009,AIRWorldwide2009
5643,{AIR Worldwide},,2010,AIRWorldwide2010,{Update to the Loss Estimates for Selected Coastal Counties},AIRWorldwide2010,AIRWorldwide2010
5118,{AIR Worldwide},,2016,AIRWorldwide2016,{2016 Global Modeled Catastrophe Losses},AIRWorldwide2016,AIRWorldwide2016


time: 125 ms (started: 2025-05-24 11:38:24 +01:00)


In [279]:
# check: no duplicates
df.loc[df.proposed_tag.duplicated(keep=False)].sort_values('proposed_tag')

,author,editor,year,tag,title,standard_tag,proposed_tag


time: 0 ns (started: 2025-05-24 11:34:04 +01:00)


In [280]:
# where has tag changewd
df.query('tag != proposed_tag')

,author,editor,year,tag,title,standard_tag,proposed_tag
8,"Goldfarb, Richard",,2010,Goldfarb2010a,{Risk-Adjusted Performance Measurement for P&C Insurers},Goldfarb2010,Goldfarb2010
9,"{Arrieta IIbarra}, Imanol and Goff, Leonard and {Jiménez Hernández}, Diego and Lainier, Jaron an...",,2018,Free2018,"{Should We Treat Data as Labor? Moving From ""free""}",ArrietaIIbarra2018,ArrietaIIbarra2018
10,"Acharya, Viral V. and Philippon, Thomas and Richardson, Matthew",,2016,Acharya,{MEASURING SYSTEMIC RISK FOR INSURANCE COMPANIES},Acharya2016,Acharya2016
19,"Shapiro, Alexander",,,Shapiro,{Errata for Lectures on Stochastic Programming},Shapiro9999,Shapiro9999
20,"Camenisch, Jan L. and Kohlweiss, Markulf and Soriente, Claudio",,2011,Leuven2011,{An accumulator based on bilinear maps and efficient revocation for anonymous credentials},Camenisch2011,Camenisch2011
...,...,...,...,...,...,...,...
6096,"Acciaio, Beatrice and Svindland, Gregor",,2013,Acciaio2013,{Are law-invariant risk functions concave on distributions?},Acciaio2013,Acciaio2013a
6102,"Hunt, Brian R.",,1994,Maps2014,{The Prevalence of Continuous Nowhere Differentiable Functions},Hunt1994,Hunt1994
6103,"Zhou, Lin",,1998,Lin1998,{Integral representation of continuous comonotonically additive functionals},Zhou1998,Zhou1998
6105,"Michel-Kerjan, Erwann and Raschky, Paul A. and Kunreuther, Howard C.",,2011,Michel-Kerjan2011a,{Corporate Demand for Insurance: An Empirical Analysis of the U.S. Market for Catastrophe and No...,MichelKerjan2011,MichelKerjan2011a


time: 15 ms (started: 2025-05-24 11:34:18 +01:00)


In [281]:
df.loc[6111]

author                                     Kuelbs, J.
editor                                               
year                                             2011
tag                                         stroock93
title           {Probability Theory An Analytic View}
standard_tag                               Kuelbs2011
proposed_tag                               Kuelbs2011
Name: 6111, dtype: object

time: 0 ns (started: 2025-05-24 11:34:50 +01:00)


In [ ]:
# what is re-mapped

In [256]:
ka = KeyAllocator(set(df.stag))

time: 16 ms (started: 2025-05-24 11:24:06 +01:00)


In [271]:
from archivum.utilities import KeyAllocator, safeyear

time: 0 ns (started: 2025-05-24 11:30:43 +01:00)


In [259]:
ka('Anewname2029')

'Anewname2029'

time: 0 ns (started: 2025-05-24 11:25:30 +01:00)


In [257]:
ka('Schmidt2003')

'Schmidt2003a'

time: 0 ns (started: 2025-05-24 11:24:15 +01:00)


In [261]:
df['proposed_tag'] = df.stag.map(ka)

time: 329 ms (started: 2025-05-24 11:26:10 +01:00)


In [263]:
df.stag

1            Schmidt2003
2          Bahnemann2015
3       Khoshnevisan2011
4              Mahul2003
5              Jiang2025
              ...       
6114           Froot2007
6115          Penner2007
6116          Davies2010
6117       Mahadevan2011
6118           Petak2000
Name: stag, Length: 6118, dtype: object

time: 0 ns (started: 2025-05-24 11:26:57 +01:00)


In [265]:
from archivum.utilities import safeyear

time: 0 ns (started: 2025-05-24 11:28:02 +01:00)


In [270]:
df.year.map(safeyear).sort_values()

4792    1871
2095    1903
1980    1914
856     1916
699     1922
        ... 
3246    9999
4550    9999
737     9999
1995    9999
573     9999
Name: year, Length: 6118, dtype: object

time: 0 ns (started: 2025-05-24 11:30:31 +01:00)


In [269]:
int('')

ValueError: invalid literal for int() with base 10: ''

time: 16 ms (started: 2025-05-24 11:29:56 +01:00)


In [268]:
df.loc[df.proposed_tag.duplicated(keep=False)].sort_values('proposed_tag')

,author,editor,year,tag,title,stag,proposed_tag
934,{Aon Benfield},,,AonBenfielda,{ReMetrica Sell Sheet},AonBenfield,AonBenfield
1094,{Aon Benfield},,,AonBenfield,{Frequently Asked Questions of new SRQ section related to ERM},AonBenfield,AonBenfield
3074,Aon plc,,,Aon,{Summary of S&P's Proposed Insurer Risk-Based Capital Adequacy Model Timing of Finalized Criteria},Aonplc,Aonplc
2655,Aon plc,,,Aonplc,{Stephen Mildenhall_TA Renewal 01 Jul 2015 to 30 June 2017.pdf},Aonplc,Aonplc
1873,"Borwein, David and Borwein, Jonathan M.",,,Wyxg,{Some Remarkable Properties of Sinc and Related Integrals},Borwein,Borwein
1445,"Borwein, Jonathan M. and Zhu, Qiji J.",,,Borwein,{A survey of subdifferential calculus with applications},Borwein,Borwein
2897,"Chen, H.",,,Chena,{Principle of Data Reduction},Chen,Chen
4388,"Chen, Ying-Erh and Liao, Shuh-Yuan",,,Chen,"{Risks 2018, 6, x; doi: FOR PEER REVIEW}",Chen,Chen
5876,"Garbarino, Ellen and Slonim, Robert and Sydnor, Justin",,,Garbarino,{Digit Ratios (2D:4D) as Predictors of Risky Decision Making for both Sexes},Garbarino,Garbarino
886,"Garbarino, Nicola and Guin, Benjamin and Lee, Jonathan",,,Garbarino2024,{The effect of subsidized flood insurance on real estate markets},Garbarino,Garbarino


time: 15 ms (started: 2025-05-24 11:28:55 +01:00)


In [258]:
df.loc[df.stag.duplicated(keep=False)].sort_values('stag').groupby('stag').apply(
    lambda x: [x.name] + [ka(x.name) for i in range(len(x) - 1)]
 )

stag
ACIA2014                                      [ACIA2014, ACIA2014a]
AMBest2005                                [AMBest2005, AMBest2005a]
AMBest2008                   [AMBest2008, AMBest2008a, AMBest2008b]
AMBest2009                   [AMBest2009, AMBest2009a, AMBest2009b]
AMBest2010      [AMBest2010, AMBest2010a, AMBest2010b, AMBest2010c]
                                       ...                         
Zhang2015                                   [Zhang2015, Zhang2015a]
Zheng2014                                   [Zheng2014, Zheng2014a]
Zhu2011                                         [Zhu2011, Zhu2011a]
Zitkovic2013                          [Zitkovic2013, Zitkovic2013a]
delaPena2007                           [delaPena2007, delaPena2007]
Length: 563, dtype: object

time: 46 ms (started: 2025-05-24 11:24:39 +01:00)


In [248]:
df .query('tag_base == tag')

,author,editor,year,tag,title,stag,tag_base
1,"Schmidt, Anatoly B.",,2003,Schmidt2003,{The true invariant of an arbitrage free portfolio},Schmidt2003,Schmidt2003
2,"Bahnemann, David",,2015,Bahnemann2015,{Distributions for Actuaries},Bahnemann2015,Bahnemann2015
3,"Khoshnevisan, Davar",,2011,Khoshnevisan2011,{Topics in Probability: Levy Processes},Khoshnevisan2011,Khoshnevisan2011
4,"Mahul, Olivier",,2003,Mahul2003,{Efficient Risk Sharing within a Catastrophe Insurance Pool},Mahul2003,Mahul2003
5,"Jiang, By John X. and Koo, David S. and Wang, Isabel Y.",,2025,Jiang2025,{The Road Not Taken: Evaluating the Consequences of the U.S.'s Non-Adoption of IFRS in 2012},Jiang2025,Jiang2025
...,...,...,...,...,...,...,...
6114,"Froot, Kenneth A.",,2007,Froot2007,"{Risk management, capital budgeting, and capital structure policy for insurers and reinsurers}",Froot2007,Froot2007
6115,"Penner, Irina",,2007,Penner2007,"{Dynamic convex risk measures: time consistency, prudence, and sustainability}",Penner2007,Penner2007
6116,"Davies, Christa and Gallant, Cliff and Shields, Meyer",,2010,Davies2010,{Q3 2010 Aon Corporation Earnings Conference Call on Oct. 29. 2010 / 12:30PM},Davies2010,Davies2010
6117,"Mahadevan, L. and Yong, Ee Hou",,2011,Mahadevan2011,"{Probability, physics, and the coin toss}",Mahadevan2011,Mahadevan2011


time: 16 ms (started: 2025-05-24 11:19:23 +01:00)


In [227]:
bit = b.df[['author', 'editor', 'year', 'tag', 'title']]

time: 0 ns (started: 2025-05-24 10:48:27 +01:00)


In [229]:
# strip off letters after year, to see if the tag has the correct form
bit['tag_base'] = bit.tag.str.replace(
    r'^(.*?[0-9]{4})[a-z]+$', r'\1', regex=True)
# suggested tag
bit['stag'] = b.suggested_tags(bit)

time: 110 ms (started: 2025-05-24 10:48:49 +01:00)


In [ ]:
# where is the tag not "official"
df = bit
fGT(df.query('tag_base != stag').sort_values('tag_base')[['tag', 'tag_base', 'stag', 'year']])

In [181]:
df['tag_proposed'] = df['stag']
# create here
ka = sc.KeyAllocator(df.tag)
# adjust where the current tag is not of the expected form 
df.loc[df.tag_base != df.stag, 'tag_proposed'] = df.loc[df.tag_base != df.stag, 'tag_proposed'].map(ka)

time: 31 ms (started: 2025-05-21 17:32:49 +01:00)


In [ ]:
df.query('tag_base != stag')[['tag', 'tag_base', 'stag', 'tag_proposed', 'author', 'title', 'year']].head(50)

In [232]:
set('a') .union( set('b'))

{'a', 'b'}

time: 16 ms (started: 2025-05-24 10:59:28 +01:00)


In [ ]:
for t in sorted(b.df.loc[b.df.tag.duplicated(keep=False), base_cols].tag.value_counts().keys()):
    bit = b.df.drop(columns=['abstract', 'annote']).query('tag == @t').T.dropna(how='all')
    bit_eq = (bit.eq(bit.iloc[:, 0], axis=0).all(axis=1)).astype(int)
    bit['union'] = bit.apply(lambda x: x.iloc[0] if pd.isna(x.iloc[1]) else x.iloc[1], axis=1)
    bit['eq'] = bit_eq
    display(fGT(bit.loc[~bit.union.isna()].fillna(''), caption=t))

# Citations
extract citations from the abstract field
see b.extract_citations()

# Windows encoding
see b.to_windows_csv

Below makes a tester.

# 

In [768]:
test = pd.DataFrame([(i, chr(i)) for i in range(65, 10000)], columns=['ord', 'char'])

time: 15 ms (started: 2025-05-20 22:44:48 +01:00)


In [770]:
test.to_csv('dm.csv', encoding='utf-8-sig')

time: 16 ms (started: 2025-05-20 22:45:16 +01:00)


In [776]:
test2 = pd.read_csv('dm2.csv', encoding='utf-8-sig', index_col=0)

time: 16 ms (started: 2025-05-20 22:49:19 +01:00)


In [777]:
test2.head()

,ord,char
0,65,A
1,66,B
2,67,C
3,68,D
4,69,E


time: 0 ns (started: 2025-05-20 22:49:20 +01:00)


In [779]:
all(test2==test)

True

time: 0 ns (started: 2025-05-20 22:49:40 +01:00)


In [772]:
fGT(test.head())

index,ord,char
0,65,A
1,66,B
2,67,C
3,68,D
4,69,E


time: 15 ms (started: 2025-05-20 22:48:29 +01:00)
